In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
import os

train_dir = "new_train"
test_dir = "new_test"

img_size = (224,224)
batch_size = 64
num_classes = 6
epochs = 20

#Base Model: MobileNetV2 for Emotion Recognition
base_model = MobileNetV2(
    input_shape=(*img_size, 3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False  # freeze feature extractor


# Build Final Model

inputs = layers.Input(shape=(*img_size, 3))
x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

model = models.Model(inputs, outputs)


# Compile Model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# ==========================
# Train Model
# ==========================
history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=test_ds
)

# ==========================
# Fine-tuning (optional)
# ==========================
# Unfreeze some deeper layers for slight fine-tuning
base_model.trainable = True
fine_tune_at = 100  # unfreeze last 30 layers

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n🔧 Fine-tuning the model...")
history_fine = model.fit(
    train_ds,
    epochs=5,
    validation_data=test_ds
)

# ==========================
# Evaluate Model
# ==========================
loss, accuracy = model.evaluate(test_ds)
print(f"\n✅ Test Accuracy: {accuracy*100:.2f}%")

# ==========================
# Save Model
# ==========================
os.makedirs("models", exist_ok=True)
model.save("models/emotion_mobilenetv2.h5")
print("\n💾 Model saved as models/emotion_mobilenetv2.h5")


Found 28273 files belonging to 6 classes.
Found 7067 files belonging to 6 classes.


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide (TrueDivide)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subtract (Subtract)             │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,726 (9.24 MB)

 Trainable params: 164,742 (643.52 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/20
311/442 ━━━━━━━━━━━━━━━━━━━━ 3:53 2s/step - accuracy: 0.2307 - loss: 1.9188

KeyboardInterrupt: 